<a href="https://colab.research.google.com/github/SmartCodar/boto3samples/blob/main/s3uploadfile.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Upload a file from local storage to AWS S3**

In [ ]:
!pip install awscli
!pip3 install boto3

In [2]:
import os
import sys
import threading
import boto3
from boto3.s3.transfer import TransferConfig

**Prepare a AWS CLI Session with Access Key and Access Secret**

In [12]:
aws_access_key_id = '<<Your aws_access_key_id>>'
aws_secret_access_key = '<<aws_secret_access_key>>'

# Configure AWS credentials
session = boto3.Session(
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key
)

In [34]:
class UploadProgressPercentage(object):
    def __init__(self, filename):
        self._filename = filename
        self._size = float(os.path.getsize(filename))
        self.session = session
        self._lock = threading.Lock()
        self._uploaded = 0

    def __call__(self, bytes_amount):
        with self._lock:
            self._uploaded += bytes_amount
            progress = (self._uploaded / self._size) * 100
            sys.stdout.write("\rUploading file %s...  %.2f%%" % (self._filename, progress))
            sys.stdout.flush()

    def upload_file(bucket_name, local_file_path, s3_file_path):
        s3 = session.client('s3')

        config = TransferConfig()
        progress = UploadProgressPercentage(local_file_path)

        s3.upload_file(
            local_file_path,
            bucket_name,
            s3_file_path,
            Config=config,
            Callback=progress
        )
        print (f'\n {filename} uploaded successfully on AWS Bucket {bucket_name}')

**Upload file to the AWS S3**

In [ ]:
bucket_name = 'karteekq3'
local_file_path = '/content/TortoiseGit-2.14.0.1-64bit.msi'
filename = os.path.basename(local_file_path)
UploadProgressPercentage.upload_file(bucket_name, local_file_path, filename)